In [30]:
import pandas as pd
import json
import tensorflow as tf
import numpy as np
from dateutil.relativedelta import *

In [14]:
transaction_csv_FILEPATH = "../data/transactions_w_districts.csv" # CHANGE this filepath appropriately
model_folder_name = "mlp_model" # CHANGE this filepath appropriately

In [2]:
trans = pd.read_csv(transaction_csv_FILEPATH, parse_dates=["Date of Sale"])
print(trans.shape)
trans.head()

(263262, 12)


C:\Users\65901\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Address/Project Name,Date of Sale,Sale Price,Private/Public,Type,Floor_min,Floor_max,Area (Sqft),Remaining Lease (2022),Flat_model,District,Price psft
0,PEOPLE'S PARK COMPLEX,2022-09-01,1060000.0,Private,Apartment,11,15,1119.45,45 years,NaN,Central Region,946.893564
1,THE SAIL @ MARINA BAY,2022-09-01,1300000.0,Private,Apartment,61,65,678.13,79 years,NaN,Central Region,1917.036556
2,MARINA ONE RESIDENCES,2022-08-01,1620000.0,Private,Apartment,26,30,699.65,88 years,NaN,Central Region,2315.443436
3,MARINA ONE RESIDENCES,2022-08-01,3744171.0,Private,Apartment,26,30,1517.71,88 years,NaN,Central Region,2466.987106
4,ONE SHENTON,2022-08-01,1100000.0,Private,Apartment,11,15,581.25,82 years,NaN,Central Region,1892.473118


In [3]:
trans.groupby([trans["Date of Sale"].dt.year, trans["Date of Sale"].dt.month])["Price psft"].count().describe()

count      69.000000
mean     3815.391304
std      1380.743121
min       592.000000
25%      3234.000000
50%      3877.000000
75%      4766.000000
max      6401.000000
Name: Price psft, dtype: float64

In [9]:
mean_df = trans.groupby([trans["Date of Sale"].dt.year, trans["Date of Sale"].dt.month]).mean()
mean_df["Date"] = list(mean_df.index)
dates = []
for d in mean_df["Date"]:
    if len(str(d[1]))<2:
        day = "0" + str(d[1])
    else:
        day = str(d[1])
    temp = str(d[0]) + "-" + day
    dates.append(temp)
mean_df["Date"] = pd.to_datetime(dates)
mean_df.reset_index(drop=True, inplace=True)
print(mean_df.shape)
mean_df.head()

(69, 4)


,Sale Price,Area (Sqft),Price psft,Date
0,427506.984810,1029.992658,420.272433,2017-01-01
1,447398.734562,1059.724175,425.967234,2017-02-01
2,445029.896942,1045.616684,429.620263,2017-03-01
3,438629.272235,1045.006427,424.037796,2017-04-01
4,443789.321212,1048.781520,426.619906,2017-05-01


In [15]:
model = tf.keras.models.load_model(model_folder_name)

In [17]:
X_final = np.array(mean_df["Price psft"][-24:]).reshape(1,24)
pred = model.predict(X_final)

In [31]:
last_date = list(mean_df["Date"])[-1]
pred_date = last_date + relativedelta(months=1)

In [32]:
pred_year = pred_date.year
pred_month = pred_date.month

index_pred = []

for i in range(12):
    month_str = str(pred_month)
    if len(month_str) < 2:
        month_str = "0" + month_str
    year_month = str(pred_year) + "-" + month_str
    index_pred.append(year_month)
    
    pred_month += 1
    if pred_month > 12:
        pred_month = 1
        pred_year += 1

In [35]:
fc = pred[0]

In [36]:
pred_json = {"x":index_pred, "y":list(fc.astype(np.float64)), "type":"scatter",
             "marker": {
                "color": "rgb(14, 37, 207)"
            },
            "line": {
                "color": "rgb(14, 37, 207)"
            }
        }

In [37]:
with open("price_pred.json", "w") as f:
    f.write(json.dumps(pred_json, indent=4))